In [2]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, List, Tuple, Any
from copy import copy

# sorting

In [3]:
def simplesort(a):
    """the simplest sort possible? arxiv.org/abs/2110.01111"""
    for i in range(len(a)):
        for j in range(len(a)):
            if a[i] < a[j]:
                a[i], a[j] = a[j], a[i]

In [4]:
def quicksort1(a):
    """a simple yet relatively fast implementation"""
    if len(a) <= 1:
        return a
    
    pivot = a[0]
    smaller = [x for x in a if x < pivot]
    equal = [x for x in a if x == pivot]
    bigger = [x for x in a if x > pivot]
    return quicksort1(smaller) + equal + quicksort1(bigger)

In [5]:
def quicksort2(a, lo=None, hi=None):
    """a more complicated but slightly faster implementation"""
    if lo is None: lo = 0
    if hi is None: hi = len(a) - 1
    
    if lo < hi:
        p = partition(a, lo, hi)
        quicksort2(a, lo, p)
        quicksort2(a, p+1, hi)

        
def partition(a, lo, hi):
    # choose a pivot
    pivot = a[(lo + hi) // 2]
    
    while True:
        # print(pivot, lo, hi)
        # find elements that need to be swapped
        while a[lo] < pivot: lo += 1
        while a[hi] > pivot: hi -= 1
            
        # check if the array is already partitioned
        # - 'hi' has to be returned here since it is the lower index and in 'quicksort2'
        #   I use quicksort2(a, lo, p), quicksort2(a, p+1, hi), so p cannot be higher than the pivot
        if lo >= hi: return hi
        
        # swap the elements
        a[lo], a[hi] = a[hi], a[lo]
        lo, hi = lo+1, hi-1

In [11]:
def counting_sort(a, n_min=0, n_max=10**4):
        a = np.array(a)
        out = np.zeros_like(a)
        counts = np.zeros(n_max-n_min+1, dtype=np.int32)
        
        # increment counter
        np.add.at(counts, a-n_min, 1)
        
        # make counter cummulative
        counts = np.cumsum(counts)
        
        # fill output array
        for n in a:
            out[counts[n-n_min]-1] = n
            counts[n-n_min] -= 1
    
        return list(out)

In [14]:
# test: correctness
n = 1_000
a = list(np.random.randint(0, n, n))
assert quicksort1(copy(a)) == sorted(a)
assert counting_sort(copy(a)) == sorted(a)
assert (simplesort(b := copy(a)) or b) == sorted(a) # inplace sorting
assert (quicksort2(b := copy(a)) or b) == sorted(a) # inplace sorting

In [15]:
# test: speed
a = list(np.random.permutation(2_000))
%time _ = simplesort(copy(a))
%time _ = quicksort1(copy(a))
%time _ = quicksort2(copy(a))
%time _ = counting_sort(copy(a))

CPU times: user 234 ms, sys: 1.88 ms, total: 236 ms
Wall time: 236 ms
CPU times: user 2.67 ms, sys: 101 µs, total: 2.77 ms
Wall time: 2.77 ms
CPU times: user 2.26 ms, sys: 0 ns, total: 2.26 ms
Wall time: 2.26 ms
CPU times: user 1.67 ms, sys: 78 µs, total: 1.75 ms
Wall time: 1.74 ms


# argsort

- if `o = argsort(a)`, then `a[o] == sorted(a)`
- if `o = argsort(argsort(a))`, then `o` is the order of elements in `a` (ie 0 means that it is the smallest element)
    - `argsort(a)[i]` is like asking *"what's the index of the `i`th smallest element in `a`?"*
    - `argsort[i]` of a shuffled array `1..n` is like asking *"what's the position of element `i` in the array?"*
    - hence, `argsort(argsort(a))[i]` is like asking *"what's the position of the `i`th element in its order, ie what's its order?"*

In [19]:
def argsort(a):
    return sorted(range(len(a)), key=a.__getitem__)

a = [1, 9, 3, 4, 7]
print(a)
print(argsort(a))
print(argsort(argsort(a)))

[1, 9, 3, 4, 7]
[0, 2, 3, 4, 1]
[0, 4, 1, 2, 3]


# binary search

In [11]:
import bisect

In [1]:
def binary_search(a, x, lo=0, hi=None):
    if hi is None: hi = len(a) - 1
    # print(lo, hi)

    mi = (lo + hi) // 2
    if a[mi] == x: return mi
    elif a[mi] < x: return binary_search(a, x, mi+1, hi)
    elif x < a[mi]: return binary_search(a, x, lo, mi-1)

In [14]:
# all array elements unique
a = np.sort(np.random.permutation(1000)[:100])
x = a[np.random.randint(0, len(a))]
assert bisect.bisect_left(a, x) == binary_search(a, x)
assert bisect.bisect_left(a, x) < bisect.bisect_right(a, x)

In [15]:
# repeating array elements
a = np.sort(np.random.randint(0, 100, 1000))
x = a[np.random.randint(0, len(a))]
print(binary_search(a, x)) # any matching index
print(bisect.bisect_left(a, x)) # leftmost matching index
print(bisect.bisect_right(a, x)) # rightmost matching index

811
808
815


In [16]:
x = [1, 2, 3]
cumsum = np.cumsum(np.sort(x))
np.where(cumsum > 2)[0][0]

1

# binary tree

In [16]:
# assuming the following implementation
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [17]:
# serialize a binary tree
def serialize(t):
    if t is None: return '.'
    return '-' + str(t.val) + serialize(t.left) + serialize(t.right)